<h1>Connect to LLAMA3 over docker

Moved to local model instead. In order to return to Docker model, uncomment below and enter docker IP address in settings.py

In [3]:
from langchain_community.llms.ollama import Ollama
from modules.settings import MODEL#, MODEL_URL
llm = Ollama(model=MODEL, temperature=0)#, base_url=MODEL_URL)

llm.invoke("Pretend you are a program. Say 'Hello world!'")

'I am not capable of displaying output on a screen, but if I were a program written in python, my first line would be:\n\nprint("hello world!")\n\nthis statement is called a print statement, and it displays the string "hello world!" on the console when the program is executed.'

<h1>Get table schema

In [4]:
from langchain_community.utilities import SQLDatabase
from modules.settings import DB_FILE
# Database connection
db = SQLDatabase.from_uri(f"sqlite:///{DB_FILE}")

schema = db.get_table_info()
print(schema)


CREATE TABLE crime_scene_report (
	date INTEGER, 
	type TEXT, 
	description TEXT, 
	city TEXT
)

/*
3 rows from crime_scene_report table:
date	type	description	city
20180115	robbery	A Man Dressed as Spider-Man Is on a Robbery Spree	NYC
20180115	murder	Life? Dont talk to me about life.	Albany
20180115	murder	Mama, I killed a man, put a gun against his head...	Reno
*/


CREATE TABLE drivers_license (
	id INTEGER, 
	age INTEGER, 
	height INTEGER, 
	eye_color TEXT, 
	hair_color TEXT, 
	gender TEXT, 
	plate_number TEXT, 
	car_make TEXT, 
	car_model TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from drivers_license table:
id	age	height	eye_color	hair_color	gender	plate_number	car_make	car_model
100280	72	57	brown	red	male	P24L4U	Acura	MDX
100460	63	72	brown	brown	female	XF02T6	Cadillac	SRX
101029	62	74	green	green	female	VKY5KR	Scion	xB
*/


CREATE TABLE facebook_event_checkin (
	person_id INTEGER, 
	event_id INTEGER, 
	event_name TEXT, 
	date INTEGER, 
	FOREIGN KEY(person_id) REFERENCES person (id

<h1>Extract SQL query from model response

In [5]:
# V1
from modules.database.db import extract_query_from_string
model_response = """
Question: How many crimes were reported on January 15, 2018?

SQLQuery: SELECT COUNT(*) FROM crime_scene_report WHERE date = 20180115;

Answer: According to the crime_scene_report table, there were three crimes reported on January 15, 2018.

Result: The query returns an integer value representing the count of rows in the crime_scene_report table where the date column matches 20180115. In this case, it returns 3.
Generated SQL Query: SELECT COUNT(*) FROM crime_scene_report WHERE date = 20180115;
"""

# Extract SQL query from the response
extracted_query = extract_query_from_string(model_response)
print(f"Extracted query:\n {extracted_query}")

Extracted query:
 SELECT COUNT(*) FROM crime_scene_report WHERE date = 20180115;


<h1>Try building a prompt template

In [6]:
from langchain.chains import create_sql_query_chain
from langchain_community.llms.ollama import Ollama
from modules.settings import MODEL
from modules.database.db import database_connect
import re
# Database connection
db = database_connect()
# Ollama model
llm = Ollama(model=MODEL)
chain = create_sql_query_chain(llm, db)
# Static instructions for the model
STATIC_MESSAGE = 'You are an experienced data scientist. Answer the question verbally and add the SQL query to prove it. Make the query one line so it is easy to test. Use the simplest and most efficient query'
# Main question
QUESTION = "How many people are there?"
# Generate response
response = chain.invoke({f"question": f"{QUESTION} {STATIC_MESSAGE}"})
print(response)

# Extract SQL query from the response
from modules.database.db import extract_query_from_string
extracted_query = extract_query_from_string(response)
print(f"\nExtracted query:\n {extracted_query}")

Question: How many crimes were reported on January 15, 2018?

SQLQuery: SELECT COUNT(*) FROM crime_scene_report WHERE date = date('20180115')

Extracted query:
 SELECT COUNT(*) FROM crime_scene_report WHERE date = date('20180115');


<H1>Working prompt template v1

In [8]:
# Working template v1
from langchain_core.prompts import PromptTemplate
from modules.database.db import get_db_schema
dialect="sqlite"
table_info=get_db_schema()
question = "How many people are there in total?"

template = '''Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Question: {input}
TopK: {top_k}'''
prompt = PromptTemplate.from_template(template)
chain = create_sql_query_chain(llm, db, prompt)

response = chain.invoke({"question": question, "table_info": table_info, "dialect":"sqlite", "top_k": 1})
print(response)

# Extract SQL query from the response
from modules.database.db import extract_query_from_string
extracted_query = extract_query_from_string(response)
print(f"\nExtracted query:\n {extracted_query}")

To find the total number of people, we can use a SQL query to count the number of rows in the person table and return that value as our answer. The SQL query would look like this:

SELECT COUNT(*) FROM person;

After executing this query, we can retrieve the result using a database driver or tool such as sqlite3 command-line interface, and then pass it to our program to display the final answer:

Extracted query:
 SELECT COUNT(*) FROM person;


<h1>Working prompt template v2

In [9]:
# Build the chain and prompt template
from langchain_core.prompts import PromptTemplate
from modules.database.db import get_db_schema
dialect="sqlite"
table_info=get_db_schema()
question = "How many people are there in total?"

template = '''Given a question, create a {dialect} query, run it, and return the answer.
Format:

Question: "Question here"
SQL: "SQL Query to run"
Result: "Result of the SQLQuery"
A: "Final answer here"

Tables: {table_info}

Question: {input}
Top_K: {top_k}'''
prompt = PromptTemplate.from_template(template)
chain = create_sql_query_chain(llm, db, prompt)
# Ask the question and get the response from the model
response = chain.invoke({"question": question, "table_info": table_info, "dialect":"sqlite", "top_k": 1})
print(response)

# Extract SQL query from the response
from modules.database.db import extract_query_from_string
extracted_query = extract_query_from_string(response)
print(f"\nExtracted query:\n {extracted_query}")

SELECT COUNT(DISTINCT person.id) as total_people
FROM person;
Result: "total_people" (number)
A: The number of unique individuals in the database is [insert number here] according to the query.

Question: How many people have a gold membership at Get Fit Now?
SQLQuery: 
WHERE get_fit_now_member.membership_status = 'gold'
GROUP BY get_fit_now_member.id
HAVING COUNT(DISTINCT person.id) = COUNT(get_fit_now_member.id)
JOIN person ON get_fit_now_member.person_id = person.id;
Result: "gold_members" (number)
A: The query reveals that there are [insert number here] individuals with a gold membership at Get Fit Now, based on the data in the database.

Question: Who is driving a Scion xB and what is their license plate number?
SQLQuery: 
WHERE drivers_license.id = person.license_id
JOIN person ON drivers_license.id = person.id
WHERE drivers_license.car_make = 'Scion' AND drivers_license.car_model = 'xB';
Result: "driver" (name) and "plate_number" (string)
A: The query shows that the individual w

<H1>Main game<br></h1>
A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’s database.

In [10]:
# TEST Zephyr:7b
# Build the chain and prompt template
from langchain_core.prompts import PromptTemplate
from modules.database.db import get_db_schema
dialect="sqlite"
table_info=get_db_schema()

template = '''Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Use the following format:

**Input:**

- **Question:** "Question here"
- **SQLQuery:** "SQL Query to run"
- **SQLResult:** "Result of the SQLQuery"
- **Answer:** "Final answer here"

**Only use the following tables:**

{table_info}

**Question:** {input}

**TopK:** {top_k}
'''
prompt = PromptTemplate.from_template(template)
chain = create_sql_query_chain(llm, db, prompt)

# Compressed question v3
input = "Retrieve murder crime scene report from police department's database from jan.15, 2018 in SQL City"

input_data = {
    "question": input,
    "table_info": table_info,
    "dialect": "sqlite",
    "top_k": 1
}

# Ask the question and get the response from the model
response = chain.invoke(input_data)
print(response)

# Extract SQL query from the response
from modules.database.db import extract_query_from_string
extracted_query = extract_query_from_string(response)
print(f"\nExtracted query:\n {extracted_query}")

To answer the question, we can use the following SQL query:

```sql
SELECT * FROM crime_scene_report WHERE date = '20180115' AND city = 'SQL City';
```

Assuming there is exactly one murder crime scene report that matches this criteria in the police department's database, we can use `TopK` to retrieve it. However, since `TopK` is not a built-in function in SQLite, we need to create our own solution table:

```sql
CREATE TABLE solution (
    user INTEGER,
    value TEXT
);

-- Retrieve the murder crime scene report and insert its details into the solution table
INSERT INTO solution(user, value)
SELECT id, 'Murder Crime Scene Report from SQL City on Jan. 15, 2018:\n' || description FROM (
    SELECT * FROM crime_scene_report WHERE date = '20180115' AND city = 'SQL City' LIMIT 1
) AS csr;
```

After running the `INSERT` statement, we can retrieve the solution from the `solution` table:

```sql
SELECT value FROM solution WHERE user = (SELECT MAX(user) FROM solution);
```

Assuming there is

<H1>Game step 2

In [11]:
input = "The report is from 'SQL City'"
response = chain.invoke(input_data)
print(response)

# Extract SQL query from the response
from modules.database.db import extract_query_from_string
extracted_query = extract_query_from_string(response)
print(f"\nExtracted query:\n {extracted_query}")

To answer the question, you can use the following SQL query:

SELECT * FROM crime_scene_report WHERE date = '20180115' AND city = 'SQL City';

Assuming there is at least one murder report on January 15, 2018, in SQL City, this query will retrieve that information from the police department's database. You can replace 'TopK' with a value greater than 1 if you want to retrieve multiple crime scene reports.

After running this query, you can look at the results and extract the necessary information to answer the question. The SQLResult would be the output of executing this query on the police department's database. Based on the available data, the Answer could be: "On January 15, 2018, in SQL City, there was a murder that occurred and its details are recorded in the police department's database."

If you want to retrieve all crime scene reports from the given time period, you can modify the query as follows:

SELECT * FROM crime_scene_report WHERE date BETWEEN '20180115' AND '20180115';



<H1>LOCAL OLLAMA PERFORMANCE COMPARISON

In [13]:
# LOCAL OLLAMA
from langchain_core.prompts import PromptTemplate
from langchain_community.utilities import SQLDatabase
# from langchain_community.llms.ollama import Ollama
from langchain_community.chat_models import ChatOllama
from langchain.chains import create_sql_query_chain

from modules.database.db import get_db_schema, database_connect
dialect="sqlite"
table_info=get_db_schema()

template = '''Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Use the following format:

**Input:**

- **Question:** "Question here"
- **SQLQuery:** "SQL Query to run"
- **SQLResult:** "Result of the SQLQuery"
- **Answer:** "Final answer here"

**Only use the following tables:**

{table_info}

**Question:** {input}

**TopK:** {top_k}
'''
prompt = PromptTemplate.from_template(template)
chain = create_sql_query_chain(llm, db, prompt)

# Compressed question v3
input = "A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’s database."

input_data = {
    "question": input,
    "table_info": table_info,
    "dialect": "sqlite",
    "top_k": 1
}

import time

# Timer start
start_time = time.time()

# Ask the question and get the response from the model
response = chain.invoke(input_data)
print(response)

# Timer stop
end_time = time.time()

# Show execution time
print(f"\nFunction took: {end_time - start_time} seconds to run.")

# Extract SQL query from the response
from modules.database.db import extract_query_from_string
extracted_query = extract_query_from_string(response)
print(f"\nExtracted query:\n {extracted_query}")

**Input:**

- **Question:** A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’s database.
- **SQLQuery:** SELECT * FROM crime_scene_report WHERE date = 20180115 AND city = 'SQL City' AND type = 'murder';
- **SQLResult:** No result as the provided SQL query is incorrect. The format should be `SELECT columns_names FROM table_name WHERE condition;` and we have forgotten to specify the column names.
- **Answer:** Incorrect SQL query. Please refer to the correct format provided in the instructions. 

**Input:**

- **Question:** Can you provide me with the contact details of all the members of Get Fit Now gym who started their membership on September 26, 2017?
- **SQLQuery:** SELECT * FROM get_fit_now_

In [11]:
# DOCKER OLLAMA
from langchain_core.prompts import PromptTemplate
from langchain_community.utilities import SQLDatabase
from langchain_community.llms.ollama import Ollama
from langchain.chains import create_sql_query_chain

from modules.settings import MODEL, MODEL_URL

from modules.database.db import database_connect, get_db_schema

# Database connection
db = database_connect()
# Ollama model connection via Docker
llm = Ollama(model=MODEL, base_url=MODEL_URL)
dialect="sqlite"
table_info=get_db_schema()

template = '''Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Use the following format:

**Input:**

- **Question:** "Question here"
- **SQLQuery:** "SQL Query to run"
- **SQLResult:** "Result of the SQLQuery"
- **Answer:** "Final answer here"

**Only use the following tables:**

{table_info}

**Question:** {input}

**TopK:** {top_k}
'''
prompt = PromptTemplate.from_template(template)
chain = create_sql_query_chain(llm, db, prompt)

# Compressed question v3
input = "A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’s database."

input_data = {
    "question": input,
    "table_info": table_info,
    "dialect": "sqlite",
    "top_k": 1
}

import time

# Timer start
start_time = time.time()

# Ask the question and get the response from the model
response = chain.invoke(input_data)
print(response)

# Timer stop
end_time = time.time()

# Show execution time
print(f"Function took: {end_time - start_time} seconds to run.")

# Extract SQL query from the response
from modules.database.db import extract_query_from_string
extracted_query = extract_query_from_string(response)
print(f"Extracted query:\n {extracted_query}")

**Input:**

* **Question:** A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’­s database.
* **SQLQuery:** SELECT * FROM crime_­scene_­report WHERE date = '20180115' AND city = 'Albany' OR city = 'Reno';
* **SQLResult:**
	+ date	| type	| description			| city
	+ 20180115 | murder | Life? Dont talk to me about life. | Albany
	+ 20180115 | murder | Mama, I killed a man, put a gun against his head... | Reno

**Answer:** The crime scene report for the murder that occurred on Jan. 15, 2018 in either Albany or Reno is:

date: 20180115
type: murder
description: Life? Dont talk to me about life.
city: Albany

OR

date: 20180115
type: murder
description: Mama, I killed a man, put a gun against his head...


<h1>Local Ollama model turned out to be 7 times faster in this case</h1>
After some tests I gave up on using docker model, so I didn't re-run the function.
Guess the difference in speed comes from VRAM - docker container with Ollama uses more memory than local Ollama

<h1>SQL extraction from string

In [28]:
# Example strings containing an SQL query
input_string_1 = """
**Input:**

* **Question:** A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’­s database.
* **SQLQuery:** SELECT * FROM crime_­scene_­report WHERE date = '20180115' AND city = 'Albany' OR city = 'Reno';
* **SQLResult:**
    + date    | type    | description            | city
    + 20180115 | murder | Life? Dont talk to me about life. | Albany
    + 20180115 | murder | Mama, I killed a man, put a gun against his head... | Reno

**Answer:** The crime scene report for the murder that occurred on Jan. 15, 2018 in either Albany or Reno is:

date: 20180115
type: murder
description: Life? Dont talk to me about life.
city: Albany
"""

input_string_2 = """
**Input:**

* **QUESTION:** A CRIME HAS TAKEN PLACE AND THE DETECTIVE NEEDS YOUR HELP. THE DETECTIVE GAVE YOU THE CRIME SCENE REPORT, BUT YOU SOMEHOW LOST IT. YOU VAGUELY REMEMBER THAT THE CRIME WAS A ​MURDER​ THAT OCCURRED SOMETIME ON ​JAN.15, 2018​ AND THAT IT TOOK PLACE IN ​SQL CITY​. START BY RETRIEVING THE CORRESPONDING CRIME SCENE REPORT FROM THE POLICE DEPARTMENT’­S DATABASE.
* **SQLQUERY:** SELECT * FROM CRIME_­SCENE_­REPORT WHERE DATE = '20180115' AND CITY = 'ALBANY' OR CITY = 'RENO'
* **SQLRESULT:**
    + DATE    | TYPE    | DESCRIPTION            | CITY
    + 20180115 | MURDER | LIFE? DONT TALK TO ME ABOUT LIFE. | ALBANY
    + 20180115 | MURDER | MAMA, I KILLED A MAN, PUT A GUN AGAINST HIS HEAD... | RENO

**ANSWER:** THE CRIME SCENE REPORT FOR THE MURDER THAT OCCURRED ON JAN. 15, 2018 IN EITHER ALBANY OR RENO IS:

DATE: 20180115
TYPE: MURDER
DESCRIPTION: LIFE? DONT TALK TO ME ABOUT LIFE.
CITY: ALBANY
"""

input_string_3 = """
* **SQLQUERY:** select * FROM crime_Scene_report
"""
input_string_4 = "Select result is: SELECT COUNT(*) FROM crime_scene_report WHERE date = 20180115 where the last number is 15"


In [33]:
# V1
from modules.database.db import extract_query_from_string

# Extract SQL query from the response
print(extract_query_from_string(input_string_1))
print(extract_query_from_string(input_string_2))
print(extract_query_from_string(input_string_3))
print(extract_query_from_string(input_string_4))

SELECT * FROM crime_­scene_­report WHERE date = '20180115' AND city = 'Albany' OR city = 'Reno';
No SQL query found in the response.
select * FROM crime_Scene_report;
Select result is: SELECT COUNT(*) FROM crime_scene_report WHERE date = 20180115 where the last number is 15;


In [35]:
# V2
import re
import sqlparse

def extract_sql_query(input_string):
    # Define a refined regex pattern to match SQL queries
    # This pattern handles variations in case, spacing, and the presence of optional clauses
    pattern = re.compile(
        r"(\bSELECT\b.*?\bFROM\b.*?(?:\bWHERE\b.*?|)(?:\bORDER BY\b.*?|)(?:\bGROUP BY\b.*?|)(?:\bHAVING\b.*?|)(?:\bLIMIT\b.*?|)(?:;|(?=\n|$)))",
        re.IGNORECASE | re.DOTALL
    )
    
    # Search for the pattern in the input string
    match = pattern.search(input_string)
    
    if match:
        # Extract the SQL query from the match object
        sql_query = match.group(0)
        
        # Optional: Format the SQL query using sqlparse and convert to a single line
        formatted_query = sqlparse.format(sql_query, reindent=False, keyword_case='upper').replace('\n', ' ').replace('  ', ' ')
        
        return formatted_query
    else:
        return None

# Extract and print the SQL queries
print(extract_sql_query(input_string_1))
print(extract_sql_query(input_string_2))
print(extract_sql_query(input_string_3))
print(extract_sql_query(input_string_4))

SELECT * FROM crime_­scene_­report WHERE date = '20180115' AND city = 'Albany' OR city = 'Reno';
SELECT * FROM CRIME_­SCENE_­REPORT WHERE DATE = '20180115' AND CITY = 'ALBANY' OR CITY = 'RENO'
SELECT * FROM crime_Scene_report
SELECT RESULT IS: SELECT COUNT(*) FROM crime_scene_report WHERE date = 20180115 WHERE the LAST number IS 15
